# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import re

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'vader_lexicon'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Normalizer
from sklearn.kernel_approximation import Nystroem
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('etl', engine)
X = df.message.values
Y = df.iloc[:,4:].copy()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    words = word_tokenize(text.lower())
    words = [w for w in words if w not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    for w in words:
        lemmatizer.lemmatize(w)
        lemmatizer.lemmatize(w, pos='v')
        lemmatizer.lemmatize(w, pos='a')
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
    ('nlp', FeatureUnion([
        ('importance', Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
         ('sentiment', VaderSentiment())
        
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('nlp', FeatureUnion(n_jobs=1,
       transformer_list=[('importance', Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...lti_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [35]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.77      1.00      0.87      4020
          1       0.00      0.00      0.00       915
          2       0.00      0.00      0.00        29
          3       0.00      0.00      0.00      2221
          4       0.00      0.00      0.00       405
          5       0.00      0.00      0.00       249
          6       0.00      0.00      0.00       140
          7       0.00      0.00      0.00        95
          8       0.00      0.00      0.00       172
          9       0.00      0.00      0.00       340
         10       0.00      0.00      0.00       580
         11       0.00      0.00      0.00       438
         12       0.00      0.00      0.00        87
         13       0.00      0.00      0.00       144
         14       0.00      0.00      0.00        52
         15       0.00      0.00      0.00       161
         16       0.00      0.00      0.00       234
         17       0.00      0.00      0.00   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#('kernel_approximation', Nystroem(random_state = 42)),
#'kernel_approximation__n_components': [100, 200],

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline = Pipeline([
    ('nlp', FeatureUnion([
        ('importance', Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize, min_df=5, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer())
        ])),
         ('sentiment', VaderSentiment(tokenize))
    ])),
    ('scale', Normalizer()),
    ('clf', MultiOutputClassifier(LinearSVC(random_state=42, class_weight='balanced')))
])


parameters = {
    'nlp__importance__tfidf__sublinear_tf': [True, False]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=2)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
cv.fit(X_train, Y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] nlp__importance__tfidf__sublinear_tf=True .......................
[CV] ........ nlp__importance__tfidf__sublinear_tf=True, total= 3.5min
[CV] nlp__importance__tfidf__sublinear_tf=True .......................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.2min remaining:    0.0s


[CV] ........ nlp__importance__tfidf__sublinear_tf=True, total= 3.6min
[CV] nlp__importance__tfidf__sublinear_tf=False ......................
[CV] ....... nlp__importance__tfidf__sublinear_tf=False, total= 3.6min
[CV] nlp__importance__tfidf__sublinear_tf=False ......................
[CV] ....... nlp__importance__tfidf__sublinear_tf=False, total= 3.6min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 21.3min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('nlp', FeatureUnion(n_jobs=1,
       transformer_list=[('importance', Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...ulti_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'nlp__importance__tfidf__sublinear_tf': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [13]:
pd.DataFrame(cv.cv_results_)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_nlp__importance__tfidf__sublinear_tf,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,110.603577,0.437512,102.343742,2.141879,True,{'nlp__importance__tfidf__sublinear_tf': True},0.281358,0.271258,0.276308,0.005050,1,0.620142,0.605137,0.612639,0.007503
1,111.011038,0.533889,105.667256,0.569938,False,{'nlp__importance__tfidf__sublinear_tf': False},0.279146,0.269431,0.274288,0.004858,2,0.619084,0.602443,0.610764,0.008321


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
Y_pred = cv.predict(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.57      0.74      0.64       866
          1       0.13      0.11      0.12        27
          2       0.70      0.73      0.72      2168
          3       0.32      0.54      0.40       373
          4       0.34      0.54      0.42       251
          5       0.22      0.35      0.27       162
          6       0.13      0.16      0.14        93
          7       0.44      0.58      0.50       161
          8       0.64      0.79      0.71       320
          9       0.71      0.81      0.76       583
         10       0.57      0.72      0.63       470
         11       0.57      0.66      0.61        86
         12       0.42      0.54      0.47       127
         13       0.19      0.21      0.20        73
         14       0.33      0.47      0.38       201
         15       0.52      0.70      0.59       215
         16       0.33      0.53      0.41       707
         17       0.20      0.38      0.27   

In [17]:
pipeline = Pipeline([
    ('nlp', FeatureUnion([
        ('importance', Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize, min_df=5, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer())
        ])),
         ('sentiment', VaderSentiment(tokenize))
    ])),
    ('scale', Normalizer()),
    ('kernel', Nystroem(random_state=42)),
    ('clf', MultiOutputClassifier(LinearSVC(random_state=42, class_weight='balanced')))
])

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('nlp', FeatureUnion(n_jobs=1,
       transformer_list=[('importance', Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...ulti_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0),
           n_jobs=1))])

In [20]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.29      0.56      0.39       917
          1       0.00      0.31      0.01        16
          2       0.53      0.48      0.51      2105
          3       0.10      0.67      0.17       423
          4       0.07      0.63      0.12       291
          5       0.04      0.51      0.07       147
          6       0.03      0.47      0.05        94
          7       0.06      0.65      0.12       153
          8       0.13      0.71      0.22       336
          9       0.37      0.65      0.47       575
         10       0.12      0.68      0.20       462
         11       0.02      0.63      0.04        79
         12       0.03      0.52      0.05        99
         13       0.02      0.48      0.04        62
         14       0.04      0.71      0.08       163
         15       0.12      0.71      0.21       222
         16       0.17      0.52      0.25       683
         17       0.08      0.72      0.15   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

class VaderSentiment(BaseEstimator, TransformerMixin):
    ''' sentiment analysis using VADER method from: 
        Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for
            Sentiment Analysis of Social Media Text. Eighth International Conference on
            Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
    '''
    
    def __init__(self, tokenizer=None):
        self.vader = SentimentIntensityAnalyzer()
        self.tokenizer = tokenizer

    def tokenize(self, text):
        text = re.sub('[^a-zA-Z0-9]', ' ', text)
        words = word_tokenize(text.lower())
        words = [w for w in words if w not in stopwords.words('english')]
        lemmatizer = WordNetLemmatizer()
        for w in words:
            lemmatizer.lemmatize(w)
            lemmatizer.lemmatize(w, pos='v')
            lemmatizer.lemmatize(w, pos='a')
        return words

    def analyze(self, text):
        if self.tokenizer is None:
            text = ' '.join(self.tokenize(text))
        else:
            text = ' '.join(self.tokenizer(text))
        scores = self.vader.polarity_scores(text)
        return pd.Series([scores['neg'], scores['neu'], scores['pos']])

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        scores = pd.Series(X).apply(self.analyze)
        scores.columns = ['neg', 'neu', 'pos']
        return scores

### 9. Export your model as a pickle file

In [21]:
joblib.dump(pipeline, 'model.pkl', compress=1)

['model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.